<a href="https://colab.research.google.com/github/Harooniqbal4879/AgenticAI/blob/main/Assignment_Solution_Building_Applications_with_LLMs%26Agents_Advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Smart Study Buddy: RAG for Your Lecture Notes with Pinecone

Welcome, aspiring AI developer! In this project, we'll build a "Smart Study Buddy." Imagine you have multiple sets of lecture notes (as text files or PDFs) for different subjects. Wouldn't it be amazing to have an intelligent assistant that can answer your questions based *only* on these notes and even remember the context of your current study session? That's exactly what we're going to build!

## The Problem: Information Overload!
As a student, you collect a lot of notes. Finding specific information quickly can be a challenge, especially when you're juggling multiple subjects. You want a tool that helps you query your own study material efficiently.

## Our Solution: A Conversational RAG System
We will build a **Retrieval Augmented Generation (RAG)** system. Here's how it will work:
1.  **Ingest Your Notes**: The Study Buddy will load lecture notes from local files (e.g., `.txt`, `.pdf`).
2.  **Create a Knowledge Base**: It will process these notes, chop them into smaller, digestible chunks, generate numerical representations (embeddings) that capture their meaning, and store these embeddings in a [Pinecone](https://www.pinecone.io/) serverless vector index. Pinecone is a specialized database designed for super-fast similarity searches on these embeddings.
3.  **Answer Questions Intelligently**: When you ask a question:
    *   The system first retrieves the most relevant snippets from your notes (thanks to Pinecone's speedy search).
    *   Then, it feeds these relevant snippets, your question, and the ongoing conversation history to a Large Language Model (LLM) like GPT.
    *   The LLM generates an answer based *specifically* on the information from your notes.
4.  **Maintain Session Context**: The Study Buddy will remember the flow of your current conversation using LangChain's memory features, allowing for natural follow-up questions.
5.  **Honest and Focused Answers**: We'll instruct the LLM to answer *only* using the provided lecture note context. If the answer isn't in your notes, the Study Buddy will say so, rather than inventing information.

## Why is this useful for students?
-   **Focused Learning**: Get answers directly from your course material, avoiding the vast (and sometimes distracting or incorrect) expanse of the general internet.
-   **Efficient Revision**: Quickly find information or clarify doubts without manually sifting through pages and pages of notes.
-   **Personalized Study Aid**: It's an AI assistant trained and focused on *your* specific learning materials.

## Prerequisites
-   A Pinecone account and API key (sign up at [pinecone.io](https://www.pinecone.io/))
-   An OpenAI API key (for the LLM and optionally for embeddings, sign up at [openai.com](https://openai.com/))
-   Python 3.8 or newer.
-   Sample lecture notes (we'll guide you to create a few `.txt` and `.pdf` files) in a local input folder.

## 1. Install Required Libraries

First, let's install the necessary Python packages. We'll need:
-   `langchain`, `langchain-community`, `langchain-openai`: Core LangChain libraries for building our RAG pipeline, document loaders, text splitters, embedding wrappers, vector store integrations, LLM integrations, and memory.
-   `pinecone-client`: The official Python client for interacting with your Pinecone vector database (version >= 3.x.x recommended for `ServerlessSpec`).
-   `sentence-transformers`: For generating high-quality sentence and text embeddings (a popular free option).
-   `openai`: The official Python client for OpenAI, used by LangChain for LLM interactions and OpenAI embeddings.
-   `pypdf`: For loading content from PDF files.
-   `python-dotenv`: For managing environment variables securely (like API keys).

Uncomment the following cell to install these packages if you haven't already.

In [ ]:
# !pip install langchain langchain-community langchain-openai pinecone-client sentence-transformers openai pypdf python-dotenv "unstructured[md,txt]"

## 2. Load Environment Variables and Initialize Core Components

We'll load our API keys from a `.env` file for security and ease of configuration.

**Action Required:** Create a file named `.env` in the same directory as this notebook with the following content:
```
PINECONE_API_KEY='YOUR_PINECONE_API_KEY'
OPENAI_API_KEY='YOUR_OPENAI_API_KEY'
```
Replace `'YOUR_PINECONE_API_KEY'` and `'YOUR_OPENAI_API_KEY'` with your actual keys.

After setting up your `.env` file, we'll import necessary modules and initialize the Pinecone client and our chosen embedding model.

In [ ]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec
from langchain_community.document_loaders import PyPDFLoader, TextLoader # Specific loaders
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings # Option 1 for embeddings
# from langchain_openai import OpenAIEmbeddings # Option 2 for embeddings (paid)
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_community.vectorstores import Pinecone as LangChainPinecone # LangChain's Pinecone integration
from langchain.prompts import PromptTemplate

# Load environment variables from .env file
load_dotenv()

# Get API keys from environment
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Verify API keys are loaded
if not PINECONE_API_KEY:
    raise ValueError("PINECONE_API_KEY not found. Please set it in your .env file or environment.")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found. Please set it in your .env file or environment.")

# Initialize Pinecone client
# This client is used for direct Pinecone operations like creating/deleting indexes
pc = Pinecone(api_key=PINECONE_API_KEY)

# Initialize Embedding Model
# We'll use HuggingFaceEmbeddings with 'all-MiniLM-L6-v2' as it's a good, free, and fast model.
# It produces 384-dimensional embeddings.
EMBEDDING_MODEL_NAME = 'all-MiniLM-L6-v2'
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)
EMBEDDING_DIMENSION = 384 # This MUST match the output dimension of EMBEDDING_MODEL_NAME

# --- OPTION 2: OpenAI Embeddings (Uncomment to use, and comment out HuggingFace above) ---
# Remember to adjust EMBEDDING_DIMENSION if you switch.
# 'text-embedding-ada-002' (1536 dims) or 'text-embedding-3-small' (1536 dims) or 'text-embedding-3-large' (3072 dims)
# embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model="text-embedding-3-small")
# EMBEDDING_DIMENSION = 1536
# print(f"Using OpenAI embedding model.")
# --- End of Option 2 ---

print(f"Successfully initialized Pinecone client and embedding model: {EMBEDDING_MODEL_NAME} (Dim: {EMBEDDING_DIMENSION}).")

## 3. Pinecone Index Setup

Our Pinecone index will be the dedicated, managed, and scalable environment for storing and searching our lecture note embeddings. If it doesn't already exist, we'll create a new **Serverless Index**. Serverless indexes are great because they automatically scale based on usage and offer a cost-effective solution by separating storage and compute.

**Key Parameters for Index Creation:**
-   `name`: A unique name for your index (e.g., `smart-study-buddy-notes`).
-   `dimension`: The dimensionality of the vectors you'll store. This **must** match the output dimension of your chosen embedding model (which we set to `EMBEDDING_DIMENSION` above).
-   `metric`: The similarity metric to use for searching. `cosine` is common for text embeddings as it measures the cosine of the angle between vectors, focusing on orientation rather than magnitude.
-   `spec`: We use `ServerlessSpec` to specify the cloud and region for our serverless index.

In [ ]:
INDEX_NAME = "smart-study-buddy-notes" # Feel free to choose a unique name
METRIC = "cosine"

try:
    # Check if the index already exists
    existing_indexes = [index_info.name for index_info in pc.list_indexes()]
    if INDEX_NAME not in existing_indexes:
        print(f"Index '{INDEX_NAME}' does not exist. Creating new serverless index...")
        pc.create_index(
            name=INDEX_NAME,
            dimension=EMBEDDING_DIMENSION,
            metric=METRIC,
            spec=ServerlessSpec(
                cloud="aws",        # You can choose "aws", "gcp", or "azure"
                region="us-east-1"  # Choose a region appropriate for you
            )
        )
        # Wait for the index to be ready (optional, but good practice for immediate use)
        import time
        while not pc.describe_index(INDEX_NAME).status['ready']:
            print("Waiting for index to be ready...")
            time.sleep(5)
        print(f"Successfully created and initialized index: '{INDEX_NAME}' with dimension {EMBEDDING_DIMENSION} and metric '{METRIC}'.")
    else:
        print(f"Using existing index: '{INDEX_NAME}'")

    # Connect to the index (this is more for direct operations, LangChain will also connect)
    pinecone_index_obj = pc.Index(INDEX_NAME)
    print(f"Successfully connected to index '{INDEX_NAME}'.")
    print(f"Index stats: {pinecone_index_obj.describe_index_stats()}")

except Exception as e:
    print(f"Error during Pinecone index setup for '{INDEX_NAME}': {str(e)}")
    raise  # Re-raise the exception to stop execution if setup fails

## 4. Data Ingestion & Processing: Your Lecture Notes

Now, let's prepare your actual lecture notes to be fed into our Study Buddy.

**ACTION REQUIRED: Prepare Your Sample Lecture Notes!**
1.  Create a directory named `input_lecture_notes` in the same location as this Jupyter notebook.
2.  Add 2-3 sample lecture note files into this directory. To test our system properly, include:
    *   At least one plain text file (e.g., `history_lecture_1.txt`).
    *   At least one PDF file (e.g., `calculus_chapter_2.pdf`).
    
    **Example Content for `history_lecture_1.txt`:**
    ```
    History of Ancient Civilizations - Lecture 1
    Key topics: Mesopotamia, Egypt, Indus Valley.
    Mesopotamia is often called the cradle of civilization. The Sumerians developed cuneiform writing.
    The Nile river was central to ancient Egyptian life, facilitating agriculture and transport.
    Pharaohs were considered divine rulers in Egypt.
    The Indus Valley Civilization, also known as the Harappan Civilization, had advanced urban planning.
    Mohenjo-daro and Harappa were major cities.
    ```
    **Example Content for `calculus_chapter_2.pdf` (Just create a simple PDF with this text):**
    ```
    Calculus I - Chapter 2: Derivatives
    The derivative of a function measures the sensitivity to change of the function value (output value) with respect to a change in its argument (input value).
    The derivative of f(x) = x^2 is f'(x) = 2x.
    The power rule is a common differentiation technique.
    Limits are fundamental to understanding derivatives.
    ```
    **Example Content for `literature_notes.txt`:**
    ```
    Introduction to Shakespearean Tragedies
    Common themes include ambition, revenge, and fate.
    Hamlet is a famous tragedy focusing on Prince Hamlet's quest for revenge.
    Key characters in Hamlet: Ophelia, Claudius, Gertrude.
    Macbeth explores the corrupting influence of unchecked ambition.
    Lady Macbeth is a powerful and manipulative character.
    ```

**The Process We'll Follow:**
1.  **Load Documents**: We'll iterate through files in the `input_lecture_notes` directory. We'll use LangChain's `TextLoader` for `.txt` files and `PyPDFLoader` for `.pdf` files to extract the text content.
2.  **Split Documents into Chunks**: Large documents often exceed the context window limits of LLMs. More importantly for RAG, smaller, focused chunks lead to more precise retrieval of relevant information. We'll use `RecursiveCharacterTextSplitter`, which tries to split text based on sensible separators (like newlines, sentences) first to keep related information together.

In [ ]:
NOTES_DIR = "./input_lecture_notes"

# Create directory if it doesn't exist (and inform the user)
if not os.path.exists(NOTES_DIR):
    os.makedirs(NOTES_DIR)
    print(f"Created input directory: {NOTES_DIR}. Please add your sample lecture note files (.txt, .pdf) to this folder.")
elif not os.listdir(NOTES_DIR):
    print(f"Input directory {NOTES_DIR} is empty. Please add your sample lecture notes to this folder for the Study Buddy to work.")

all_documents = []
if os.path.exists(NOTES_DIR) and os.listdir(NOTES_DIR):
    print(f"Loading documents from {NOTES_DIR}...")
    for filename in os.listdir(NOTES_DIR):
        filepath = os.path.join(NOTES_DIR, filename)
        try:
            if filename.endswith(".pdf"):
                loader = PyPDFLoader(filepath)
                loaded_docs = loader.load()
                print(f"- Loaded PDF: {filename} ({len(loaded_docs)} page(s))")
                all_documents.extend(loaded_docs)
            elif filename.endswith(".txt"):
                loader = TextLoader(filepath, encoding='utf-8') # Specify encoding for robustness
                loaded_docs = loader.load()
                print(f"- Loaded TXT: {filename} ({len(loaded_docs)} document(s))")
                all_documents.extend(loaded_docs)
            else:
                print(f"- Skipping unsupported file type: {filename}")
        except Exception as e:
            print(f"Error loading file {filename}: {e}")

    if all_documents:
        print(f"\nSuccessfully loaded content from {len(all_documents)} source(s) (pages/files).")
        # print(f"First document content preview (first 200 chars): {all_documents[0].page_content[:200]}")
        # print(f"First document metadata: {all_documents[0].metadata}")
    else:
        print("No processable documents were found or loaded. Please check the NOTES_DIR and file types.")
else:
    print(f"Input directory '{NOTES_DIR}' is missing or empty. Skipping document loading.")

# Initialize text splitter
# RecursiveCharacterTextSplitter tries to split based on a list of characters (e.g., "\n\n", "\n", " ", "").
# chunk_size: The maximum size of a chunk (in characters, by default).
# chunk_overlap: The number of characters to overlap between chunks to maintain context.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # Max characters per chunk
    chunk_overlap=200,    # Characters of overlap between chunks
    length_function=len   # How to measure chunk size (using len() for characters)
)

# Split documents into chunks
chunks = []
if all_documents:
    print(f"\nSplitting {len(all_documents)} document(s)/page(s) into chunks...")
    chunks = text_splitter.split_documents(all_documents)
    print(f"Successfully created {len(chunks)} chunks from the documents.")
    if chunks:
        print(f"First chunk content preview (first 100 chars): '{chunks[0].page_content[:100]}...'" )
        print(f"First chunk metadata: {chunks[0].metadata}")
else:
    print("No documents were loaded, so no chunks to create.")

## 5. Embedding & Vector Store: Building the Knowledge Base in Pinecone

With our lecture notes loaded and thoughtfully chunked, the next critical step is to convert these text chunks into **embeddings**. Recall that embeddings are numerical vector representations that capture the semantic meaning of the text. Texts with similar meanings will have embeddings that are close together in the vector space.

We will then store these embeddings in our Pinecone index (`smart-study-buddy-notes`). LangChain's `PineconeVectorStore.from_documents` (which we imported as `LangChainPinecone`) simplifies this process significantly. It takes our list of `chunks`, uses the `embeddings` model we initialized earlier (e.g., `HuggingFaceEmbeddings`), generates the embeddings, and then uploads (upserts) them into our specified Pinecone index.

**This is the step where we populate Pinecone, making your notes searchable for the RAG system!** If you re-run this notebook and your notes haven't changed, you might see Pinecone report that vectors are already present from a previous run. Pinecone stores this data persistently.

In [ ]:
vectorstore = None # Initialize to None, will be our LangChainPinecone vector store object

if chunks:
    print(f"\nGenerating embeddings for {len(chunks)} chunks and storing them in Pinecone index '{INDEX_NAME}'...")
    # This might take some time depending on the number of chunks and your internet connection.
    try:
        # This command does the following:
        # 1. Takes each Document chunk.
        # 2. Uses the `embeddings` model (HuggingFaceEmbeddings) to create a vector for it.
        # 3. Upserts (uploads/inserts) the vector and its associated text/metadata to the Pinecone index.
        vectorstore = LangChainPinecone.from_documents(
            documents=chunks,          # The LangChain Document chunks
            embedding=embeddings,      # The initialized HuggingFace embedding function
            index_name=INDEX_NAME      # The name of your Pinecone index
        )
        print(f"Successfully stored embeddings in Pinecone index '{INDEX_NAME}'.")

        # It might take a few moments for Pinecone's stats to update after upserting.
        import time
        time.sleep(10) # Give Pinecone a moment to update stats
        stats = pc.Index(INDEX_NAME).describe_index_stats()
        print(f"\nUpdated Index Statistics for '{INDEX_NAME}':")
        print(f"Total vectors: {stats.total_vector_count}")
        print(f"Namespaces: {stats.namespaces}")

    except Exception as e:
        print(f"Error storing embeddings in Pinecone: {e}")
        # If 'vectorstore' couldn't be created from new documents,
        # try to connect to an existing index if it might already have data.
        print(f"Attempting to connect to existing index '{INDEX_NAME}' for retriever initialization.")
        try:
            vectorstore = LangChainPinecone.from_existing_index(index_name=INDEX_NAME, embedding=embeddings)
            print("Successfully connected to existing index for retriever.")
            stats_existing = pc.Index(INDEX_NAME).describe_index_stats()
            if stats_existing.total_vector_count > 0:
                print(f"Existing index contains {stats_existing.total_vector_count} vectors.")
            else:
                print("Warning: Connected to existing index, but it appears to be empty. Q&A might not work well.")
        except Exception as e_existing:
            print(f"Could not connect to or use existing index '{INDEX_NAME}': {e_existing}")
            vectorstore = None # Ensure it's None if connection failed
else:
    print("\nNo chunks were created from documents. Trying to connect to an existing Pinecone index for Q&A...")
    # If no new chunks, we must rely on an existing index that might have data from previous runs.
    try:
        vectorstore = LangChainPinecone.from_existing_index(index_name=INDEX_NAME, embedding=embeddings)
        print(f"Successfully connected to existing index '{INDEX_NAME}'.")
        stats = pc.Index(INDEX_NAME).describe_index_stats()
        if stats.total_vector_count == 0:
            print(f"Warning: Index '{INDEX_NAME}' exists but is empty. The Study Buddy won't have any notes to refer to.")
        else:
            print(f"Index contains {stats.total_vector_count} vectors from previous uploads.")
    except Exception as e:
        print(f"Could not connect to existing index '{INDEX_NAME}'. The Study Buddy will not function without a populated vector store: {e}")
        vectorstore = None

## 6. Building the "Brain": Question Answering Chain with Session Memory

This is where the magic happens! We'll construct the core logic of our Smart Study Buddy using LangChain's `ConversationalRetrievalChain`. This chain is specifically designed for Q&A over documents while maintaining a conversation history.

Here's what it does:
1.  **Takes User's Question**: You type in a question.
2.  **Considers Chat History**: It looks at the past turns in the conversation (if any) to understand the context. It might even rephrase your current question based on the history to make it a better standalone query for retrieval.
3.  **Retrieves Relevant Documents**: It uses the `vectorstore` (our Pinecone index) as a `retriever` to find the document chunks most similar (relevant) to your (possibly rephrased) question.
4.  **Constructs a Prompt**: It takes your original question, the retrieved document chunks (as context), and feeds them to an LLM.
5.  **Generates an Answer**: The LLM (e.g., `ChatOpenAI` with a GPT model) generates a response based on the question and the provided context from your notes.
6.  **Updates Memory**: The new question and answer are added to the `ConversationBufferMemory` for future turns.

### Customizing the LLM's Behavior with a Prompt Template
To ensure our Study Buddy behaves as intended (answers only from notes, indicates if info is missing), we'll use a `PromptTemplate`. This template will instruct the LLM on how to formulate its answers.

**Key Components:**
-   **LLM**: We'll use `ChatOpenAI`. `temperature=0` makes the output more deterministic and factual.
-   **Retriever**: Our `vectorstore` (Pinecone) will be converted into a retriever.
-   **Memory**: `ConversationBufferMemory` will store the chat history.
-   **Prompt Template**: To guide the LLM.

In [ ]:
qa_chain = None
chat_history = [] # Initialize an empty list for chat history (for the chain's memory)

if vectorstore:
    # Initialize the LLM
    llm = ChatOpenAI(
        temperature=0,  # Lower temperature for more factual, less creative responses
        model="gpt-3.5-turbo", # A good balance of capability and cost. Or use "gpt-4-turbo-preview" or "gpt-4"
        openai_api_key=OPENAI_API_KEY
    )
    print(f"LLM initialized using model: {llm.model_name}")

    # Create a retriever from our Pinecone vector store
    retriever = vectorstore.as_retriever(
        search_type="similarity", # Other options: "mmr" (Maximal Marginal Relevance)
        search_kwargs={"k": 3}    # Retrieve top 3 most relevant chunks
    )
    print(f"Retriever created from Pinecone index '{INDEX_NAME}', retrieving top 3 chunks.")

    # Define the prompt template
    # This tells the LLM how to behave. It should use the provided context (from your notes)
    # and the chat history to answer the question.
    prompt_template_str = """
    You are a helpful AI Smart Study Buddy. Use the following pieces of context from lecture notes and the chat history to answer the question at the end.
    Your goal is to answer the user's question based *only* on the provided lecture notes context.
    Do not use any external knowledge or make up information.
    If the answer to the question cannot be found in the provided context, clearly state "I'm sorry, but I couldn't find information about that in your lecture notes."
    If the context is empty or irrelevant to the question, also state that you cannot find the answer in the notes.

    Context from lecture notes:
    {context}

    Chat History:
    {chat_history}

    Question: {question}
    Helpful Answer from your lecture notes:
    """
    QA_PROMPT = PromptTemplate(template=prompt_template_str, input_variables=["context", "chat_history", "question"])
    print("Prompt template defined.")

    # Initialize conversation memory
    # `return_messages=True` ensures the memory object returns messages in the format expected by the chain.
    # `memory_key='chat_history'` is the default and matches the input variable in ConversationalRetrievalChain.
    memory = ConversationBufferMemory(
        memory_key='chat_history',
        return_messages=True,
        output_key='answer' # Ensure the LLM's answer is stored correctly in memory for the next turn
    )
    print("Conversation memory initialized.")

    # Create the Conversational Retrieval Chain
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory,
        return_source_documents=True,  # To see which chunks were retrieved
        combine_docs_chain_kwargs={"prompt": QA_PROMPT}, # Pass our custom prompt
        verbose=False # Set to True for more detailed logging from the chain
    )
    print("Conversational Retrieval Chain created successfully.")
else:
    print("Vectorstore (Pinecone connection) not available. Skipping Conversational Retrieval Chain setup.")
    print("Please ensure documents are loaded and embeddings are stored in Pinecone.")

## 7. Chatting with Your Smart Study Buddy!

Now for the exciting part: interacting with your Study Buddy! We'll create a simple function to send questions to our `qa_chain` and display the answers along with the source document snippets that the LLM used.

**We will test the following scenarios:**
1.  **Initial Question**: Ask a question that can be answered from one of your sample lecture notes.
2.  **Follow-up Question**: Ask a question that relies on the context of the previous question/answer within the same session (testing the memory).
3.  **Question Not in Documents**: Ask a question whose answer is very unlikely to be in your sample notes (testing the LLM's adherence to the prompt to only use provided context).

In [ ]:
def chat_with_buddy(query: str):
    global chat_history # Uses the global chat_history list managed by the chain's memory
    if not qa_chain:
        print("QA chain is not initialized. Cannot process query. Please check previous steps.")
        return "Error: QA chain not set up."

    try:
        print(f"\n🤔 User Query: {query}")

        # Invoke the chain. It uses the `memory` object which contains `chat_history`.
        # The `chat_history` argument to `invoke` here is for passing explicit history if not using memory's implicit handling,
        # but with `ConversationBufferMemory`, the chain manages it internally.
        # For `ConversationalRetrievalChain`, the input is a dict with "question" and "chat_history".
        # The memory object automatically provides the `chat_history`.
        result = qa_chain.invoke({"question": query}) # `chat_history` is implicitly handled by the memory object

        answer = result["answer"]
        print(f"💡 Study Buddy: {answer}")

        # The memory object updates chat_history automatically.
        # We can inspect it if needed: print(f"Current memory: {qa_chain.memory.buffer_as_messages}")

        print("\n📚 Sources Used:")
        if result.get("source_documents"):
            for i, doc in enumerate(result["source_documents"]):
                source_name = doc.metadata.get('source', 'Unknown source')
                # Truncate page_content for display
                content_preview = doc.page_content.replace('\n', ' ').strip()[:150]
                print(f"  {i+1}. Source: {source_name}\n     Content Preview: '{content_preview}...'")
        else:
            print("  No specific source documents were heavily relied upon or returned by the retriever.")

        return answer
    except Exception as e:
        error_message = f"Error during chat interaction: {str(e)}"
        print(error_message)
        # If there's an error, you might want to log it or display a more user-friendly message.
        # Also, consider what to do with chat_history in case of an error.
        # For simplicity, we'll just return the error message.
        return error_message

# --- Test Scenarios ---
if qa_chain: # Ensure the chain is ready
    print("--- Starting Chat Session with Smart Study Buddy ---")

    # 1. Initial Question (assuming you have notes about history or calculus based on sample data)
    # Adjust this question based on the content of YOUR sample notes!
    # Example questions based on the sample notes provided earlier:
    # question1 = "What was cuneiform writing developed by?"
    # question1 = "What is the derivative of x^2?"
    question1 = "What are common themes in Shakespearean tragedies?"
    response1 = chat_with_buddy(question1)

    # 2. Follow-up Question (relies on the context of the previous Q/A)
    if response1 and "Error:" not in response1 and "I'm sorry" not in response1:
        # Adjust this follow-up based on your question1 and expected answer.
        # question2 = "Tell me more about Mesopotamia."
        # question2 = "What is the power rule related to?"
        question2 = "Can you name a key character in Hamlet?"
        response2 = chat_with_buddy(question2)
    else:
        print("\nSkipping follow-up question as the first answer was not found or an error occurred.")

    # 3. Question Not in Documents
    question3 = "What is the airspeed velocity of an unladen swallow?" # Unlikely to be in lecture notes
    response3 = chat_with_buddy(question3)

    question4 = "What is the capital of France?" # General knowledge, should not be answered from notes.
    response4 = chat_with_buddy(question4)

    print("\n--- Chat Session Ended ---")
    print("\n--- Note on Session Memory vs. Persistent Knowledge ---")
    print("The 'chat_history' (session memory) is for the current run of this notebook.")
    print("If you restart the kernel and re-run, the Study Buddy won't remember this specific conversation.")
    print("However, the knowledge (document embeddings) stored in your Pinecone index IS persistent.")
    print("So, on a new run, it can still answer questions about your notes, but it starts a fresh conversation.")

else:
    print("\nSkipping chat example as the QA chain is not set up. Please check previous steps.")
    print("Ensure you have documents in 'input_lecture_notes', API keys are correct, and Pinecone index is accessible.")

## 8. Conclusion and Next Steps

Congratulations! You've successfully built a "Smart Study Buddy" – a conversational RAG pipeline that uses Pinecone to query your personal lecture notes and LangChain to manage the conversation flow and LLM interaction.

You've learned how to:
1.  Set up Pinecone and initialize a serverless vector index.
2.  Load and process local documents (`.txt`, `.pdf`) using LangChain document loaders.
3.  Chunk documents effectively for RAG.
4.  Generate embeddings for these chunks and store them in your Pinecone index.
5.  Create a `ConversationalRetrievalChain` with session memory (`ConversationBufferMemory`).
6.  Use a `PromptTemplate` to guide the LLM to answer based *only* on the provided context and to indicate if the answer isn't in the notes.
7.  Test your RAG system with initial questions, follow-up questions, and questions designed to check its adherence to the provided context.

### Possible Enhancements and Further Exploration:
-   **More Document Types**: Extend to support `.docx`, `.pptx`, etc., using appropriate LangChain loaders (often via `UnstructuredFileLoader`).
-   **Different Embedding Models/LLMs**: Experiment with other embedding models (e.g., OpenAI's `text-embedding-3-small`) or different LLMs (`gpt-4-turbo-preview`, models from Hugging Face Hub) to see their impact. Remember to adjust `EMBEDDING_DIMENSION` if your embedding model changes!
-   **Advanced Retrieval**: Explore techniques like Maximal Marginal Relevance (MMR) search in the retriever, or re-ranking retrieved documents before sending them to the LLM.
-   **Metadata Filtering**: Add metadata to your document chunks during ingestion (e.g., subject, lecture number, date) and modify the retriever to filter based on this metadata for more targeted searches (e.g., "What did we learn about photosynthesis in Biology lecture 3?").
-   **More Sophisticated Memory**: For very long conversations, explore `ConversationSummaryMemory` or `ConversationSummaryBufferMemory` which summarize older parts of the conversation to save tokens.
-   **User Interface**: Build a web interface using Streamlit or Gradio to make your Study Buddy more accessible.
-   **Evaluation**: Implement metrics to evaluate the quality of retrieval (e.g., hit rate) and generation (e.g., RAGAs, faithfulness, relevance).

## 9. Important: Clean Up Pinecone Resources

Pinecone serverless indexes incur costs based on storage and usage. If you are done with this demo and don't plan to use the index further, remember to **delete your Pinecone index** to avoid ongoing charges.

You can do this via the Pinecone console (go to [app.pinecone.io](https://app.pinecone.io/), find your index, and delete it) or programmatically by uncommenting and running the cell below. **Make sure `INDEX_NAME` matches the index you want to delete!**

In [ ]:
# # Ensure 'pc' (Pinecone client) and 'INDEX_NAME' are correctly defined from earlier cells.
# try:
#     if INDEX_NAME in [index_info.name for index_info in pc.list_indexes()]:
#         print(f"Deleting index '{INDEX_NAME}'...")
#         pc.delete_index(INDEX_NAME)
#         print(f"Index '{INDEX_NAME}' deleted successfully.")
#     else:
#         print(f"Index '{INDEX_NAME}' not found. No deletion needed or wrong index name specified.")
# except Exception as e:
#     print(f"Error deleting index '{INDEX_NAME}': {e}")